Other model tests


- from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
- from sklearn.neighbors import KNeighborsClassifier
- from sklearn.linear_model import LogisticRegression
- import xgboost as xgb
- from sklearn.tree import DecisionTreeClassifier


- Ensemble methods



In [100]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split, KFold

import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from scipy import stats
import glob
import time

In [4]:
training_src=[]
test_src = []
validation_src = []

In [5]:
file_list = glob.glob("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/*.txt")
for file in file_list:
    f_type = file.split("_pred_")[-1].split("_")[0]
    if f_type == "train":
        training_src.append(file)
    elif f_type == "val":
        validation_src.append(file)
    elif f_type == "test":
        test_src.append(file)

In [6]:
"""
training_src = []
validation_src = []
test_src = []
for x in range(0,100):
    training_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_train_with_target_cut_data.txt".format(x))
    validation_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_val_with_target_cut_data.txt".format(x))
    test_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_test_with_target_cut_data.txt".format(x))

"""

'\ntraining_src = []\nvalidation_src = []\ntest_src = []\nfor x in range(0,100):\n    training_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_train_with_target_cut_data.txt".format(x))\n    validation_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_val_with_target_cut_data.txt".format(x))\n    test_src.append("351_Data/CIFAR_100_filtered/Intermediate Data from Conv layers/{}_pred_test_with_target_cut_data.txt".format(x))\n\n'

In [7]:
def read_data(src):
    with open(src, 'r') as myfile:
        data=myfile.read().replace('\n', '')
    return data

In [8]:
def treat_data(data_string):
    data_string = data_string.replace("]", "")
    data_string = data_string.replace(" ", "")
    data_split = data_string.split("[")
    data_split = [(d.split(",")) for d in data_split]
    data_split_clean = [d[:-1] for d in data_split[:-1]]
    data_split_clean.append(data_split[-1])
    data_split_clean = [d for d in data_split_clean if d]
    return data_split_clean

In [9]:
def split_data(data):
    inp = []
    target = []
    for d in data:
        inp.append([float(x) for x in d[:-1]])
        target.append(float(d[-1]))
    inp = np.array(inp)
    target = np.array(target)
    return inp,target

In [10]:
def get_data_from_source (src):
    data_string = read_data(src)
    data_split = treat_data(data_string)
    return split_data(data_split)

In [11]:
def evaluate(model, data):
    y_pred = model.predict(data[0])
    try: predictions = [round(value) for value in y_pred]
    except: predictions = [np.argmax(value) for value in y_pred]
    # evaluate predictions
    accuracy = accuracy_score(data[1], predictions)
    return accuracy

In [12]:
def get_measures(measure, data_type="data", measure_name="Accuracy"):
    return "%s in %s: %.2f%%" % (measure_name, data_type, measure * 100.0)

In [13]:
def get_data_from_sources (training_src, test_src, validation_src=""):
    train = get_data_from_source(training_src)
    test = get_data_from_source(test_src)
    try:
        if validation_src !="":
            val = get_data_from_source(validation_src)
        else:
            inp_test, inp_val, target_test, target_val = train_test_split(*test)
            test = (inp_test, target_test)
            val = (inp_val, target_val)
    except: 
        inp_test, inp_val, target_test, target_val = train_test_split(*test)
        test = (inp_test, target_test)
        val = (inp_val, target_val)
    return train, val, test

In [14]:
train_data = []
test_data = []
val_data = []

ind = 0
for ind, _ in enumerate(training_src):
    if ind>=len(validation_src):
        val_src = ""
    else:
        val_src = validation_src[ind]
    train, val, test = get_data_from_sources(training_src[ind], test_src[ind], val_src)
    train_data.append(train)
    test_data.append(test)
    val_data.append(val)

In [15]:
num_classes = int(max(train_data[0][1])+1)

In [16]:
mlp_models = []

In [17]:
model = Sequential()
model.add(Dense(1024, input_shape=(2048,)))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

mlp_models.append(model)

In [18]:
model = Sequential()
model.add(Dense(512, input_shape=(2048,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

mlp_models.append(model)

In [19]:
model = Sequential()
model.add(Dense(num_classes, input_shape=(2048,)))
model.add(Activation('softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

mlp_models.append(model)

In [20]:
for model in mlp_models:
    
epochs=5

Crossvalidation of neural networks done using mlp_models with multiple model architectures in list

In [21]:
parameter_candidates = {}

In [107]:
svm_candidates = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  # {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

In [108]:
rfe_candidates = [
  {'n_estimators': [10, 100, 1000], 'criterion': ['gini', "entropy"]}
]

In [109]:
adb_candidates = [
  {'n_estimators': [10, 100, 1000], 'learning_rate': [1.0, 0.5, 0.1], 'algorithm':["SAMME.R", "SAMME"]}
]

In [110]:
gbc_candidates = [
  {'n_estimators': [10, 100, 1000], 'learning_rate': [0.05, 0.1, 0.5], 'criterion':["friedman_mse", "mae"]}
]

In [111]:
lr_candidates= [
    {'penalty': ["l2", "l5"]}
]

In [112]:
dtc_candidates= [
    {'criterion': ["l1", "l2"], 'splitter':['best']}
]

In [113]:
xgbc_candidates = [
    {
        'max_depth':[10], 'learning_rate':[0.1], 'n_estimators':[1000]     # objective, booster
    }
]

In [114]:
parameter_candidates ["svm"] = svm_candidates
parameter_candidates["rfe"] = rfe_candidates
parameter_candidates ["adb"] = adb_candidates
parameter_candidates["gbc"] = gbc_candidates
parameter_candidates["lr"] = lr_candidates
parameter_candidates ["dtc"] = dtc_candidates
parameter_candidates["xgbc"] = xgbc_candidates

In [115]:
rfe = RandomForestClassifier()
adb = AdaBoostClassifier()
gbc = GradientBoostingClassifier()
lr = LogisticRegression()
dtc = DecisionTreeClassifier()#
xgbc = XGBClassifier()
svc = svm.SVC()
# Genetic Programming-based

# benchmark_models = {"rfe":rfe, "xgbc":xgbc, "adb":adb, "gbc":gbc, "lr":lr, "dtc":dtc, "svm":svc}
benchmark_models = {"adb":adb, "gbc":gbc, "lr":lr, "dtc":dtc, "svm":svc}
for x in range(len(mlp_models)):
    benchmark_models["mlp_{}".format(x)] = mlp_models[x] 

In [116]:
def grid_search_model(benchmark_models, parameter_candidates, train, val, test):
    for model_key in benchmark_models.keys():
        model = benchmark_models[model_key]
        print ((" Model: " + str(model_key)+ " ").center(30, '#'))
        try:
            t0 = time.time()
            clf = GridSearchCV(estimator=model, param_grid=parameter_candidates[model_key], n_jobs=-1, cv=4)
            clf.fit (*train)
            t1 = time.time()
            total = t1-t0
            print("Training Time: {}".format(total))
            print (clf.score(*train))
            print (clf.score(*val))
            print (clf.score(*test))
            print (clf.best_estimator_)
            benchmark_models[model_key] = clf.best_estimator_
        except ValueError:
            history = model.fit(train[0], to_categorical(train[1]), epochs = epochs, validation_data = (val[0], to_categorical(val[1])))
            benchmark_models[model_key] = model
    return benchmark_models


In [ ]:
grid_search_model(benchmark_models, parameter_candidates, train_data[0], val_data[0], test_data[0])

######### Model: adb #########


In [499]:
def train_model (benchmark_models, train, val):
    for model in benchmark_models.values():
        try: model.fit(*train)
        except ValueError:
            history = model.fit(train[0], to_categorical(train[1]), epochs = epochs, validation_data = (val[0], to_categorical(val[1])))

In [ ]:
def evaluate_model(benchmark_models, train, val, test):
    for model_key in benchmark_models.keys():
        try:
            model = benchmark_models[model_key]
            train_eval = evaluate(model,train)
            val_eval = evaluate(model, val)
            test_eval = evaluate(model, test)
            print ((" Model: " + str(model_key)+ " ").center(30, '#'))
            print (get_measures(train_eval, "Train", "Accuracy"))
            print (get_measures(val_eval, "Validation", "Accuracy"))    
            print (get_measures(test_eval, "Test", "Accuracy"))    
        except Exception as ex:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print (message)

In [69]:
def train_and_evaluate_model (benchchmark_models, train, val, test, model_iterations=10):
    model_unseen_measures = {}
    for model_key in benchmark_models.keys():
        model_unseen_measures[model_key] = []
        for i in range(model_iterations):
            try:
                model = benchmark_models[model_key]
                try: 
                    model.fit(*train)
                except ValueError:
                    history = model.fit(train[0], to_categorical(train[1]), epochs = epochs, validation_data = (val[0], to_categorical(val[1])))
                train_eval = evaluate(model,train)
                val_eval = evaluate(model, val)
                test_eval = evaluate(model, test)
                model_unseen_measures[model_key].append(test_eval)
                
                print ((" Model: " + str(model_key)+ " ").center(30, '#'))
                print (get_measures(train_eval, "Train", "Accuracy"))
                print (get_measures(val_eval, "Validation", "Accuracy"))    
                print (get_measures(test_eval, "Test", "Accuracy"))
            except Exception as ex:
                template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                message = template.format(type(ex).__name__, ex.args)
                print (message)
                model_unseen_measures[model_key].append("NaN")

    return model_unseen_measures

In [70]:
def test_results (model_results):
    p_values = {}
    for m_key in model_results.keys():
        cp_values = []
        ref_model = model_results[m_key]
        for m2_key in model_results.keys():
            comp_model = model_results[m2_key]
            # pv = test_significance(ref_model, comp_model)
            statistic, p_value = stats.ttest_ind(ref_model, comp_model)
            if statistic >= 0:
                cp_values.append(p_value)
            else:
                cp_values.append(-p_value)
        p_values [m_key] = cp_values
    pd_res = pd.DataFrame(p_values, index=model_results.keys())
    pd_res = pd_res[list(model_results.keys())]
    return pd_res

In [35]:
for ind, _ in enumerate (train_data):
    print (("").center(60, '_'))
    print (("").center(60, '#'))
    print ((" Dataset Index: " + str(ind)+ " ").center(60, '#'))
    print (("").center(60, '#'))
    # train_model(benchmark_models, train_data[ind], val_data[ind])
    # evaluate_model(benchmark_models, train_data[ind], val_data[ind], test_data[ind])
    model_results = train_and_evaluate_model(benchmark_models, train_data[ind], val_data[ind], test_data[ind])
    test_pd = test_results (model_results)
    test_pd

____________________________________________________________
############################################################
##################### Dataset Index: 0 #####################
############################################################
######### Model: rfe #########
Accuracy in Train: 99.84%
Accuracy in Validation: 10.24%
Accuracy in Test: 52.30%
######### Model: rfe #########
Accuracy in Train: 99.81%
Accuracy in Validation: 11.52%
Accuracy in Test: 52.00%
######### Model: rfe #########
Accuracy in Train: 99.79%
Accuracy in Validation: 9.84%
Accuracy in Test: 52.90%
######### Model: rfe #########
Accuracy in Train: 99.68%
Accuracy in Validation: 8.32%
Accuracy in Test: 50.90%
######### Model: rfe #########
Accuracy in Train: 99.81%
Accuracy in Validation: 8.56%
Accuracy in Test: 50.80%
######### Model: rfe #########
Accuracy in Train: 99.84%
Accuracy in Validation: 10.96%
Accuracy in Test: 51.80%
######### Model: rfe #########
Accuracy in Train: 99.71%
Accuracy in Validation: 

3750/3750 [==============================] - 8s 2ms/step - loss: 0.8148 - acc: 0.7347 - val_loss: 4.0640 - val_acc: 0.0824
Epoch 2/5
3750/3750 [==============================] - 8s 2ms/step - loss: 0.1461 - acc: 0.9563 - val_loss: 4.1838 - val_acc: 0.0888
Epoch 3/5
3750/3750 [==============================] - 8s 2ms/step - loss: 0.0664 - acc: 0.9835 - val_loss: 4.2793 - val_acc: 0.0896
Epoch 4/5
3750/3750 [==============================] - 8s 2ms/step - loss: 0.0348 - acc: 0.9941 - val_loss: 4.3550 - val_acc: 0.0968
Epoch 5/5
3750/3750 [==============================] - 8s 2ms/step - loss: 0.0256 - acc: 0.9979 - val_loss: 4.4615 - val_acc: 0.0936
######## Model: mlp_0 ########
Accuracy in Train: 100.00%
Accuracy in Validation: 9.36%
Accuracy in Test: 72.20%
Train on 3750 samples, validate on 1250 samples
Epoch 1/5
3750/3750 [==============================] - 8s 2ms/step - loss: 0.0190 - acc: 0.9968 - val_loss: 4.5088 - val_acc: 0.0928
Epoch 2/5
3750/3750 [==============================

Train on 3750 samples, validate on 1250 samples
Epoch 1/5
3750/3750 [==============================] - 5s 1ms/step - loss: 1.3011 - acc: 0.5811 - val_loss: 4.1749 - val_acc: 0.0984
Epoch 2/5
3750/3750 [==============================] - 4s 1ms/step - loss: 0.4005 - acc: 0.8749 - val_loss: 4.2026 - val_acc: 0.1128
Epoch 3/5
3750/3750 [==============================] - 4s 1ms/step - loss: 0.2037 - acc: 0.9435 - val_loss: 4.2839 - val_acc: 0.1112
Epoch 4/5
3750/3750 [==============================] - 4s 1ms/step - loss: 0.1241 - acc: 0.9667 - val_loss: 4.3834 - val_acc: 0.1160
Epoch 5/5
3750/3750 [==============================] - 4s 1ms/step - loss: 0.0849 - acc: 0.9816 - val_loss: 4.4042 - val_acc: 0.1240
######## Model: mlp_1 ########
Accuracy in Train: 100.00%
Accuracy in Validation: 12.40%
Accuracy in Test: 74.40%
Train on 3750 samples, validate on 1250 samples
Epoch 1/5
3750/3750 [==============================] - 4s 1ms/step - loss: 0.0569 - acc: 0.9888 - val_loss: 4.4150 - val_acc:

3750/3750 [==============================] - 4s 1ms/step - loss: 9.8646e-04 - acc: 1.0000 - val_loss: 5.1229 - val_acc: 0.1392
######## Model: mlp_1 ########
Accuracy in Train: 100.00%
Accuracy in Validation: 13.92%
Accuracy in Test: 75.10%
Train on 3750 samples, validate on 1250 samples
Epoch 1/5
3750/3750 [==============================] - 1s 204us/step - loss: 1.9321 - acc: 0.3645 - val_loss: 3.8042 - val_acc: 0.1256
Epoch 2/5
3750/3750 [==============================] - 0s 77us/step - loss: 0.8955 - acc: 0.7205 - val_loss: 3.8351 - val_acc: 0.1232
Epoch 3/5
3750/3750 [==============================] - 0s 77us/step - loss: 0.5673 - acc: 0.8424 - val_loss: 3.8678 - val_acc: 0.1264
Epoch 4/5
3750/3750 [==============================] - 0s 76us/step - loss: 0.4010 - acc: 0.9048 - val_loss: 3.8927 - val_acc: 0.1232
Epoch 5/5
3750/3750 [==============================] - 0s 77us/step - loss: 0.3015 - acc: 0.9365 - val_loss: 3.9182 - val_acc: 0.1296
######## Model: mlp_2 ########
Accuracy 

Epoch 4/5
3750/3750 [==============================] - 0s 102us/step - loss: 0.0039 - acc: 1.0000 - val_loss: 4.7116 - val_acc: 0.1312
Epoch 5/5
3750/3750 [==============================] - 0s 103us/step - loss: 0.0036 - acc: 1.0000 - val_loss: 4.7293 - val_acc: 0.1320
######## Model: mlp_2 ########
Accuracy in Train: 100.00%
Accuracy in Validation: 13.20%
Accuracy in Test: 73.90%


NameError: name 'test_significance' is not defined

In [ ]:
def test_significance(conf1, conf2, sig_level=0.05):
    """performs a statistical significance test on the passed lists, treating each list as coming from one distribution

        Args:
            conf1: list of first mse
            conf2: list of second mse
            sig_level: significance level to test for

        Returns:
            list of mse values from letting model run

    """
    statistic, p_value = stats.ttest_ind(conf1,conf2)
    if p_value<sig_level:
        if statistic>0:
            return conf2
        else:
            return conf1
    else:
        return False

- Save outputs to file
- Run each model multiple times to get statistical significance
- Automatically test for statistical significances

In [68]:
test_pd = test_results (model_results)
test_pd

C:\Users\jonat\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\jonat\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\jonat\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,rfe,xgbc,adb,gbc,lr,dtc,svm,mlp_0,mlp_1,mlp_2
rfe,1.000000e+00,1.797128e-24,-6.067143e-24,8.260538e-24,1.934962e-25,-1.265404e-17,1.275433e-26,3.055909e-24,7.245837e-25,4.428094e-19
xgbc,-1.797128e-24,NaN,-0.000000e+00,-2.929703e-23,4.285574e-257,-1.142595e-26,6.140541e-264,2.299097e-12,8.784935e-17,3.869942e-05
adb,6.067143e-24,0.000000e+00,NaN,4.681320e-48,1.366934e-278,1.154758e-12,2.921167e-279,1.818783e-34,8.985480e-35,1.415555e-24
gbc,-8.260538e-24,2.929703e-23,-4.681320e-48,1.000000e+00,5.157499e-31,-2.827154e-26,5.919476e-36,4.431050e-16,4.389186e-19,1.099647e-07
lr,-1.934962e-25,-4.285574e-257,-1.366934e-278,-5.157499e-31,1.000000e+00,-2.894083e-27,5.140132e-257,-5.945087e-02,1.263954e-09,8.375546e-01
dtc,1.265404e-17,1.142595e-26,-1.154758e-12,2.827154e-26,2.894083e-27,1.000000e+00,4.968795e-28,1.805255e-26,7.225133e-27,3.061831e-22
svm,-1.275433e-26,-6.140541e-264,-2.921167e-279,-5.919476e-36,-5.140132e-257,-4.968795e-28,1.000000e+00,-2.556793e-16,-2.049406e-11,-1.352906e-06
mlp_0,-3.055909e-24,-2.299097e-12,-1.818783e-34,-4.431050e-16,5.945087e-02,-1.805255e-26,2.556793e-16,1.000000e+00,2.101365e-08,4.680721e-01
mlp_1,-7.245837e-25,-8.784935e-17,-8.985480e-35,-4.389186e-19,-1.263954e-09,-7.225133e-27,2.049406e-11,-2.101365e-08,1.000000e+00,-1.032150e-02
mlp_2,-4.428094e-19,-3.869942e-05,-1.415555e-24,-1.099647e-07,-8.375546e-01,-3.061831e-22,1.352906e-06,-4.680721e-01,1.032150e-02,1.000000e+00


In [66]:
pd.DataFrame(model_results)

,adb,dtc,gbc,lr,mlp_0,mlp_1,mlp_2,rfe,svm,xgbc
0,0.339,0.386,0.690,0.73,0.722,0.744,0.689,0.523,0.765,0.705
1,0.339,0.397,0.690,0.73,0.722,0.752,0.726,0.520,0.765,0.705
2,0.339,0.383,0.689,0.73,0.730,0.743,0.735,0.529,0.765,0.705
3,0.339,0.405,0.690,0.73,0.729,0.749,0.736,0.509,0.765,0.705
4,0.339,0.404,0.691,0.73,0.724,0.743,0.737,0.508,0.765,0.705
5,0.339,0.380,0.690,0.73,0.725,0.744,0.739,0.518,0.765,0.705
6,0.339,0.389,0.691,0.73,0.730,0.743,0.736,0.512,0.765,0.705
7,0.339,0.378,0.691,0.73,0.734,0.746,0.736,0.521,0.765,0.705
8,0.339,0.388,0.690,0.73,0.732,0.738,0.737,0.507,0.765,0.705
9,0.339,0.391,0.691,0.73,0.725,0.751,0.739,0.511,0.765,0.705
